In [2]:
# !pip install kr8s

In [3]:
!pip list | grep kr8s

kr8s                                     0.12.10


In [4]:
import kr8s

# for node in kr8s.get("nodes"):
#     print(node.name)

k3d-syft-dev-server-0


In [ ]:
# !mkdir serviceaccount
# kubectl get secret worker-manager-token -o jsonpath="{.data.token}" | base64 --decode > serviceaccount/token
# kubectl get secret worker-manager-token -o jsonpath='{.data.ca\.crt}' | base64 --decode > serviceaccount/ca.crt

In [105]:
type(pod)

kr8s.objects.Pod

In [7]:
import kr8s

client = kr8s.api(
    serviceaccount="./serviceaccount"
)

In [8]:
client.get("nodes")

[<Node k3d-syft-dev-server-0>]

In [22]:
from kr8s.objects import Pod

backend_pod = Pod.get("backend-0")

In [23]:
backend_pod

<Pod backend-0>

In [ ]:
# new_pod = backend_pod.copy()
# new_pod['metadata']['name'] = "worker"
# new_pod['metadata'].pop('resourceVersion', None)  # Remove the resourceVersion
# new_pod['metadata'].pop('uid', None)  # Remove the UID
# new_pod['metadata'].pop('creationTimestamp', None)  # Remove the creationTimestamp
# new_pod['metadata'].pop('selfLink', None)  # Remove the selfLink
# new_pod['metadata'].pop('ownerReferences', None)  # Remove owner references if any

In [ ]:
# # Create the new pod
# client.create(body=new_pod, namespace="syft")

In [27]:
backend_pod.name

'backend-0'

In [29]:
backend_pod.namespace

'syft'

In [44]:
s = backend_pod.spec

In [34]:
m = backend_pod.metadata

In [ ]:
backend_pod

'backend-68bd474464'

In [37]:
m.to_dict()["labels"]

{'app.kubernetes.io/component': 'backend',
 'app.kubernetes.io/managed-by': 'Helm',
 'app.kubernetes.io/name': 'devspace-app',
 'controller-revision-hash': 'backend-68bd474464',
 'statefulset.kubernetes.io/pod-name': 'backend-0'}

In [28]:
backend_pod.metadata

Box({'name': 'backend-0', 'generateName': 'backend-', 'namespace': 'syft', 'uid': '396905bb-865a-48e7-8998-473eccd819c4', 'resourceVersion': '1562', 'creationTimestamp': '2023-12-19T04:59:04Z', 'labels': {'app.kubernetes.io/component': 'backend', 'app.kubernetes.io/managed-by': 'Helm', 'app.kubernetes.io/name': 'devspace-app', 'controller-revision-hash': 'backend-68bd474464', 'statefulset.kubernetes.io/pod-name': 'backend-0'}, 'ownerReferences': [{'apiVersion': 'apps/v1', 'kind': 'StatefulSet', 'name': 'backend', 'uid': 'c613f592-967d-4997-bfb4-8e21654a2a8f', 'controller': True, 'blockOwnerDeletion': True}], 'managedFields': [{'manager': 'k3s', 'operation': 'Update', 'apiVersion': 'v1', 'time': '2023-12-19T04:59:04Z', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:metadata': {'f:generateName': {}, 'f:labels': {'.': {}, 'f:app.kubernetes.io/component': {}, 'f:app.kubernetes.io/managed-by': {}, 'f:app.kubernetes.io/name': {}, 'f:controller-revision-hash': {}, 'f:statefulset.kubernetes.io/pod-

In [55]:
image = s.containers[0].image
image

'k3d-registry.localhost:12345/openmined/grid-backend:dev-latest'

In [56]:
env = s.containers[0].env
env

BoxList([{'name': 'MONGO_PORT', 'value': '27017'}, {'name': 'MONGO_HOST', 'value': 'mongo'}, {'name': 'MONGO_USERNAME', 'value': 'root'}, {'name': 'MONGO_PASSWORD', 'valueFrom': {'secretKeyRef': {'name': 'mongosecret', 'key': 'mongo-password-secret'}}}, {'name': 'SERVICE_NAME', 'value': 'backend'}, {'name': 'RELEASE', 'value': 'production'}, {'name': 'VERSION', 'value': 'dev-latest'}, {'name': 'VERSION_HASH', 'value': 'abc'}, {'name': 'NODE_TYPE', 'value': 'domain'}, {'name': 'NODE_NAME', 'value': 'mynode'}, {'name': 'NODE_SIDE_TYPE', 'value': 'high'}, {'name': 'STACK_API_KEY', 'valueFrom': {'secretKeyRef': {'name': 'syftsecret', 'key': 'stackApiKey'}}}, {'name': 'PORT', 'value': '80'}, {'name': 'IGNORE_TLS_ERRORS', 'value': 'False'}, {'name': 'HTTP_PORT', 'value': '80'}, {'name': 'HTTPS_PORT', 'value': '443'}, {'name': 'QUEUE_PORT', 'value': '5556'}, {'name': 'CREATE_PRODUCER', 'value': 'true'}, {'name': 'N_CONSUMERS', 'value': '0'}, {'name': 'CONTAINER_HOST', 'value': 'k8s'}, {'name'

In [70]:
new_env = {}
for item in env:
    d = item.to_dict()
    print(d)
    if "value" in d:
        new_env[d["name"]] = d["value"]
    else:
        new_env[d["name"]] = "missing"

{'name': 'MONGO_PORT', 'value': '27017'}
{'name': 'MONGO_HOST', 'value': 'mongo'}
{'name': 'MONGO_USERNAME', 'value': 'root'}
{'name': 'MONGO_PASSWORD', 'valueFrom': {'secretKeyRef': {'name': 'mongosecret', 'key': 'mongo-password-secret'}}}
{'name': 'SERVICE_NAME', 'value': 'backend'}
{'name': 'RELEASE', 'value': 'production'}
{'name': 'VERSION', 'value': 'dev-latest'}
{'name': 'VERSION_HASH', 'value': 'abc'}
{'name': 'NODE_TYPE', 'value': 'domain'}
{'name': 'NODE_NAME', 'value': 'mynode'}
{'name': 'NODE_SIDE_TYPE', 'value': 'high'}
{'name': 'STACK_API_KEY', 'valueFrom': {'secretKeyRef': {'name': 'syftsecret', 'key': 'stackApiKey'}}}
{'name': 'PORT', 'value': '80'}
{'name': 'IGNORE_TLS_ERRORS', 'value': 'False'}
{'name': 'HTTP_PORT', 'value': '80'}
{'name': 'HTTPS_PORT', 'value': '443'}
{'name': 'QUEUE_PORT', 'value': '5556'}
{'name': 'CREATE_PRODUCER', 'value': 'true'}
{'name': 'N_CONSUMERS', 'value': '0'}
{'name': 'CONTAINER_HOST', 'value': 'k8s'}
{'name': 'TRACE', 'value': 'False'}


In [78]:
new_env["N_CONSUMERS"] = "1"
new_env["CREATE_PRODUCER"] = "False"

In [84]:
new_env["MONGO_PASSWORD"] = "mongosecret"
new_env["STACK_API_KEY"] = "syftsecret"
new_env["DEFAULT_ROOT_PASSWORD"] = "syftsecret"
new_env["S3_ROOT_PWD"] = "seaweedfsecret"

In [85]:
new_env

{'MONGO_PORT': '27017',
 'MONGO_HOST': 'mongo',
 'MONGO_USERNAME': 'root',
 'MONGO_PASSWORD': 'mongosecret',
 'SERVICE_NAME': 'backend',
 'RELEASE': 'production',
 'VERSION': 'dev-latest',
 'VERSION_HASH': 'abc',
 'NODE_TYPE': 'domain',
 'NODE_NAME': 'mynode',
 'NODE_SIDE_TYPE': 'high',
 'STACK_API_KEY': 'syftsecret',
 'PORT': '80',
 'IGNORE_TLS_ERRORS': 'False',
 'HTTP_PORT': '80',
 'HTTPS_PORT': '443',
 'QUEUE_PORT': '5556',
 'CREATE_PRODUCER': 'False',
 'N_CONSUMERS': '1',
 'CONTAINER_HOST': 'k8s',
 'TRACE': 'False',
 'JAEGER_HOST': 'localhost',
 'JAEGER_PORT': '14268',
 'DEV_MODE': 'False',
 'DOMAIN_CONNECTION_PORT': '3030',
 'ENABLE_OBLV': 'false',
 'DEFAULT_ROOT_EMAIL': 'info@openmined.org',
 'DEFAULT_ROOT_PASSWORD': 'syftsecret',
 'S3_ROOT_USER': 'admin',
 'S3_ROOT_PWD': 'seaweedfsecret',
 'S3_PORT': '8333',
 'SEAWEED_MOUNT_PORT': '4001'}

In [89]:
# for k, v in new_env.items():
#     print("k", k, v, type(k), type(v))
#     assert isinstance(k, str), isinstance(v, str)

In [93]:
import copy

In [94]:
new_env = copy.deepcopy(env)

In [100]:
for item in new_env:
    if item["name"] == "N_CONSUMERS":
        item["value"] = "1"
    if item["name"] == "CREATE_PRODUCER":
        item["value"] = "False"

In [101]:
new_env

BoxList([{'name': 'MONGO_PORT', 'value': '27017'}, {'name': 'MONGO_HOST', 'value': 'mongo'}, {'name': 'MONGO_USERNAME', 'value': 'root'}, {'name': 'MONGO_PASSWORD', 'valueFrom': {'secretKeyRef': {'name': 'mongosecret', 'key': 'mongo-password-secret'}}}, {'name': 'SERVICE_NAME', 'value': 'backend'}, {'name': 'RELEASE', 'value': 'production'}, {'name': 'VERSION', 'value': 'dev-latest'}, {'name': 'VERSION_HASH', 'value': 'abc'}, {'name': 'NODE_TYPE', 'value': 'domain'}, {'name': 'NODE_NAME', 'value': 'mynode'}, {'name': 'NODE_SIDE_TYPE', 'value': 'high'}, {'name': 'STACK_API_KEY', 'valueFrom': {'secretKeyRef': {'name': 'syftsecret', 'key': 'stackApiKey'}}}, {'name': 'PORT', 'value': '80'}, {'name': 'IGNORE_TLS_ERRORS', 'value': 'False'}, {'name': 'HTTP_PORT', 'value': '80'}, {'name': 'HTTPS_PORT', 'value': '443'}, {'name': 'QUEUE_PORT', 'value': '5556'}, {'name': 'CREATE_PRODUCER', 'value': 'False'}, {'name': 'N_CONSUMERS', 'value': '1'}, {'name': 'CONTAINER_HOST', 'value': 'k8s'}, {'name

In [102]:
pod = Pod.gen(name="worker", image=image, env=new_env)

In [103]:
pod.create()

In [106]:
type(pod)

kr8s.objects.Pod

In [109]:
dir(pod)

['__await__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_asyncio',
 '_exec',
 '_exists',
 '_patch',
 '_raw',
 '_refresh',
 '_set_owner',
 '_test_conditions',
 '_watch',
 'adopt',
 'annotate',
 'annotations',
 'api',
 'create',
 'delete',
 'endpoint',
 'exec',
 'exists',
 'gen',
 'get',
 'keys',
 'kind',
 'label',
 'labels',
 'list',
 'logs',
 'metadata',
 'name',
 'namespace',
 'namespaced',
 'patch',
 'plural',
 'portforward',
 'raw',
 'ready',
 'refresh',
 'replicas',
 'scalable',
 'scalable_spec',
 'scale',
 'set_owner',
 'singular',
 'spec',
 'status',
 'to_dict',
 'to_lightkube',
 'to_pykube',
 'version',
 'wait',
 'watch']

In [115]:
search_uid = pod.metadata.uid

In [124]:
pods = kr8s.get("pods", namespace=kr8s.ALL)

In [125]:
pods

[<Pod coredns-77ccd57875-rpqvz>,
 <Pod local-path-provisioner-957fdf8bc-wvl4d>,
 <Pod helm-install-traefik-crd-hxb2z>,
 <Pod helm-install-traefik-48b7g>,
 <Pod svclb-traefik-1524c6db-pbm8k>,
 <Pod metrics-server-648b5df564-zmm9v>,
 <Pod traefik-64f55bb67d-9zrzx>,
 <Pod proxy-569ffd4765-fczzc>,
 <Pod frontend-7cb7885db7-b799x>,
 <Pod backend-0>,
 <Pod seaweedfs-0>,
 <Pod mongo-0>,
 <Pod worker>]

In [126]:
found_pod = None
for pod in pods:
    if pod.metadata.uid == search_uid:
        print("found", pod)
        found_pod = pod

found worker


In [129]:
found_pod.delete()

In [128]:
dir(found_pod)

['__await__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_asyncio',
 '_exec',
 '_exists',
 '_patch',
 '_raw',
 '_refresh',
 '_set_owner',
 '_test_conditions',
 '_watch',
 'adopt',
 'annotate',
 'annotations',
 'api',
 'create',
 'delete',
 'endpoint',
 'exec',
 'exists',
 'gen',
 'get',
 'keys',
 'kind',
 'label',
 'labels',
 'list',
 'logs',
 'metadata',
 'name',
 'namespace',
 'namespaced',
 'patch',
 'plural',
 'portforward',
 'raw',
 'ready',
 'refresh',
 'replicas',
 'scalable',
 'scalable_spec',
 'scale',
 'set_owner',
 'singular',
 'spec',
 'status',
 'to_dict',
 'to_lightkube',
 'to_pykube',
 'version',
 'wait',
 'watch']

In [ ]:
# Filter pods by UID
matching_pod = next((pod for pod in all_pods if pod['metadata']['uid'] == search_uid), None)

if matching_pod:
    print("Found Pod:", matching_pod['metadata']['name'], "in Namespace:", matching_pod['metadata']['namespace'])
else:
    print("No pod found with UID", search_uid)